In [258]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()
import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo
using LinearAlgebra
# using Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using MuJoCo
using Distributions

  Activating project at `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Project.toml`
  No Changes to `~/Conan/piano project/ocrl-piano/Julia_fmincon_tests/Manifest.toml`


In [259]:
# include fmincon for ipopt
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [271]:
# import piano model
model = load_model("piano_with_one_shadow_hand_small/scene.xml")
data = init_data(model)

MuJoCo Data object

Model info

In [272]:
println("number of geoms: ",model.ngeom)
println("number of controls: ",model.nu)
println("number of states: ",model.nv+model.nq)
step!(model, data)
println("Simulation timestep: ", model.opt.timestep)
println("Positions of joints: ", data.qpos)
println("Cartesion position of joints: ", data.geom_xpos)
println("Size of Cartesion position of joints: ", size(data.geom_xpos))
println("model ")
finger_geom_indices = [42,50,58,68,77]
"""
index finger:
    name:
        rh_shadow_hand//unnamed_geom_25
        rh_shadow_hand//unnamed_geom_26
    index:
        42
        43
    range:
        x:
        y:-0.6-0.5
        z:0-0.3

middle finger:
    name:
        rh_shadow_hand//unnamed_geom_33
        rh_shadow_hand//unnamed_geom_34
    index:
        50
        51
ring finger:
    name:
        rh_shadow_hand//unnamed_geom_41
        rh_shadow_hand//unnamed_geom_42
    index:
        58
        59
pinky finger:
    name:
        rh_shadow_hand//unnamed_geom_51
        rh_shadow_hand//unnamed_geom_52
    index:
        68
        69
thumb:
    name:
        rh_shadow_hand//unnamed_geom_60
        rh_shadow_hand//unnamed_geom_61
    index:
        77
        78
""";

number of geoms: 79
number of controls: 7
number of states: 46
Simulation timestep: 0.005
Positions of joints: [-8.823595343596229e-5; -0.0004997467505765748; -8.823595343596229e-5; -8.823595343596229e-5; -0.0004997467505765748; -8.823595343596229e-5; -0.0004997467505765748; -8.823595343596229e-5; -8.823595343596229e-5; -0.0004997467505765748; -8.823595343596229e-5; -0.0004997467505765748; -8.823595343596229e-5; -0.0004997467505765748; -8.823595343596229e-5; -0.0024722821415270805; -1.4991067233811733e-5; -0.013804142838231965; 0.0014856115890729772; -0.0004493568635900772; 0.00038700999023048106; 9.237873394442637e-5; 2.1318187519532408e-5;;]
Cartesion position of joints: [0.0 0.0 0.0; -0.127 0.0 0.02; 0.0 -0.094 0.01125; -0.03 -0.08224999999999999 0.026000000000000002; 0.0 -0.07050000000000001 0.01125; 0.0 -0.047 0.01125; -0.03 -0.03474999999999999 0.026000000000000002; 0.0 -0.023499999999999993 0.01125; -0.03 -0.011249999999999996 0.026000000000000002; 0.0 0.0 0.01125; 0.0 0.0234999

Visualize target finger position

In [273]:
# control indices:
# 1: wrist left right
# 2: wrist up down
# 8, 9, 10: index finger joints
# 21: whole hand left right 
# 22: whole hand front back

In [276]:
reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension

states = zeros(nx, 10)
states[:,1] = get_physics_state(model, data)
U = zeros(model.nu)
U[4] = 0.664
U[5] = 1.1
U[6] = -0.145
# U[1] = -0.234
for t in 1:9
    data.ctrl .= U
    step!(model, data)
    println(data.geom_xpos[finger_geom_indices[1],:])
    states[:,t+1] = get_physics_state(model, data)
end
# println(data.geom_xpos[finger_geom_indices[1],:])
# reset!(model, data)

# println(data.geom_xpos[finger_geom_indices[1],:])
# data.ctrl .= U
# data.qpos .= states[:,1][1:model.nq]
# data.qvel .= states[:,1][model.nq+1:end]
# forward!(model, data)
# println("hi")
# println(data.geom_xpos[finger_geom_indices[1],:])
init_visualiser()
visualise!(model, data, trajectories = [states])

[0.0004900000000000147, 0.11699999999999999, 0.12000000000000001]
[9.420228302373101e-5, 0.11707858372791452, 0.11829545723459366]
[-0.0002552130414641713, 0.11680210004510828, 0.11550524031226599]
[-0.0002997330396959808, 0.11595516508144302, 0.11209378436064316]
[3.921388805463305e-5, 0.11444321298069857, 0.1083946695174555]
[0.0007352545522538462, 0.11226821557168055, 0.10464188922833728]
[0.0017034956743104505, 0.10950792293429841, 0.10099058179403336]
[0.002863504562663294, 0.1062517391522313, 0.09753993759777424]
[0.004144879673323398, 0.10259570024338777, 0.09434766079808861]
 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.


## Test DIRCOL: moving index finger left right

In [358]:
function wrapped_mj_step(model, data, x, uk, forward)
    # given the current model and data. set the state and control to the model and perform a forward step
    # data.ctrl .= zeros(model.nu)
    if typeof(x) == Vector{Float64}
        # set control 
        data.ctrl[:] .= uk
        # set state
        # data.qpos .= x[1:model.nq]
        # data.qvel .= x[(model.nq + 1):end]
        data.qpos .= x[1:model.nq]
        data.qvel .= x[model.nq+1:end]
    else
        # if using diff types, we need to convert the dual numbers to floats
        converted_uk = ForwardDiff.value.(uk)
        converted_x = ForwardDiff.value.(x)
        # set control
        data.ctrl[:] .= converted_uk

        # set state
        data.qpos .= converted_x[1:model.nq]
        data.qvel .= converted_x[(model.nq + 1):end]
    end
    
    # take discrete dynamics step 
    if forward
        forward!(model,data)
    else
        step!(model, data) 
    end

    # return updated state k + 1
    # xkp1 = zeros(model.nq + model.nv) 
    # xkp1 .= get_physics_state(model, data)
    # finger_coordinates = data.geom_xpos[finger_geom_indices,:]
    
    return data.geom_xpos[finger_geom_indices[1],:], get_physics_state(model, data)
end

function robohand_cost(params::NamedTuple, Z::Vector)::Real
    # TODO: implement cost function
    idx, N, xg = params.idx, params.N, params.xg
    model = params.model
    data = init_data(model)
    step!(model, data)
    Q, R, R_d = params.Q, params.R, params.R_d

    # stage cost
    cost = 0.0
    xi = Z[idx.x[1]]
    ui = Z[idx.u[1]]

    finger_geom_xpos, _ = wrapped_mj_step(model, data, xi, ui, true)
    cost += 0.5*(finger_geom_xpos-xg)'*Q*(finger_geom_xpos-xg)+0.5*ui'*R*ui
    for i = 2:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]]
        ui_1 = Z[idx.u[i-1]]
        finger_geom_xpos, _ = wrapped_mj_step(model, data, xi, ui, true)
        # @show xi-xg
        # cost += 0.5*(xi[94:96]-xg[94:96])'*Q*(xi[94:96]-xg[94:96])+0.5*ui'*R*ui
        cost += 0.5*(finger_geom_xpos-xg)'*Q*(finger_geom_xpos-xg)+0.5*ui'*R*ui+0.5*(ui-ui_1)'*R_d*(ui-ui_1)
    end

    # # terminal cost 
    # xf = Z[idx.x[N]]
    # # cost += 0.5*(xf[94:96]-xg[94:96])'*Qf*(xf[94:96]-xg[94:96])
    # finger_geom_xpos, _ = wrapped_mj_step(model, data, xf, 0, false)
    # cost += 0.5*(finger_geom_xpos-xg)'*Qf*(finger_geom_xpos-xg)

    return cost
end
function robohand_dynamics_constraints(params::NamedTuple, Z::Vector)::Vector
    idx, N, dt = params.idx, params.N, params.dt
    model = params.model
    data = init_data(model)
    step!(model, data)
    # create c in a ForwardDiff friendly way (check HW0)
    c = zeros(eltype(Z), idx.nc)
    
    for i = 1:(N-1)
        xi = Z[idx.x[i]]
        ui = Z[idx.u[i]] 
        xip1 = Z[idx.x[i+1]]
        
        _,  xip1_mujoco = wrapped_mj_step(model, data, xi, ui, false)

        c[idx.c[i]] = xip1_mujoco - xip1
    end
    # println(typeof(c))
    return c 
end

function robohand_equality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement equality constraints
    # return zeros(eltype(Z), 0)
    N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    model = params.model
    data = params.data
    
    # initial_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[1]], Z[idx.u[1]], true)
    terminal_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[N-1]], Z[idx.u[N-1]] , true)
    

    con_1 =  Z[idx.x[1]] - xic
    
    con_2 =  terminal_pose_index - xg

    return [con_1; con_2; robohand_dynamics_constraints(params, Z)]
    # return zeros(eltype(Z), 0)
end
function robohand_inequality_constraints(params::NamedTuple, Z::Vector)::Vector
    # TODO: implement inequality constraints
    # println(Z)
    # return zeros(eltype(Z), 0)
    # N, idx, xic, xg = params.N, params.idx, params.xic, params.xg 
    # model = params.model
    # data = params.data
    
    # # initial_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[1]], Z[idx.u[1]], true)
    # terminal_pose_index,_ = wrapped_mj_step(model, data, Z[idx.x[N-1]], Z[idx.u[N-1]] , true)
    

    # con_1 =  Z[idx.x[1]] - xic
    
    # con_2 =  terminal_pose_index - xg

    # return [con_1; con_2; robohand_dynamics_constraints(params, Z)]
    return zeros(eltype(Z), 0)
end
function create_idx(nx,nu,N)
    # This function creates some useful indexing tools for Z 
    # x_i = Z[idx.x[i]]
    # u_i = Z[idx.u[i]]
    
    # Feel free to use/not use anything here.
    
    
    # our Z vector is [x0, u0, x1, u1, …, xN]
    nz = (N-1) * nu + N * nx # length of Z 
    x = [(i - 1) * (nx + nu) .+ (1 : nx) for i = 1:N]
    u = [(i - 1) * (nx + nu) .+ ((nx + 1):(nx + nu)) for i = 1:(N - 1)]
    
    # constraint indexing for the (N-1) dynamics constraints when stacked up
    c = [(i - 1) * (nx) .+ (1 : nx) for i = 1:(N - 1)]
    nc = (N - 1) * nx # (N-1)*nx 
    
    return (nx=nx,nu=nu,N=N,nz=nz,nc=nc,x= x,u = u,c = c)
end

create_idx (generic function with 1 method)

In [398]:

function solve_finger_moving(;verbose=true)
    # instantiate model and data
    model = load_model("piano_with_one_shadow_hand_small/scene.xml")
    data = init_data(model)

    # reset the model and data
    reset!(model, data)

    # initiate time and time steps
    dt = 0.01
    tf = 0.1
    t_vec = 0:dt:tf
    N = length(t_vec)

    # LQR cost
    # Q size of full states
    # Q = diagm(ones(model.nq + model.nv))
    # R = 0.1*diagm(ones(model.nu))
    # Qf = 10*diagm(ones(model.nq + model.nv))

    Q = 100*diagm(ones(3))
    # R = 0.1*diagm(ones(model.nu))
    R = 0.1*diagm(ones(model.nu))
    R_d = 10*diagm(ones(model.nu))

    idx = create_idx(model.nq + model.nv, model.nu, N)
    # initial and goal states
    step!(model, data)
    xic = get_physics_state(model, data)
    # xic = data.geom_xpos[finger_geom_indices[1],:]
    U = zeros(model.nu)
    U[4] = 0.664
    U[5] = 1.1
    U[2] = 0.21
    for t in 1:9
        data.ctrl .= U
        step!(model, data)

    end
    # initial and goal state
    xg_physics = get_physics_state(model, data)
    xg = data.geom_xpos[finger_geom_indices[1],:]
    xg[3] = xg[3]
    # reset the model and data
    reset!(model, data)
    step!(model, data)
    params = (Q = Q, R = R, R_d = R_d, xic = xic, xg = xg, dt = dt, N = N, idx = idx, model=model, data=data)

    # primal bounds
    x_l = -1 * Inf * ones(idx.nz)
    x_u = Inf*ones(idx.nz)
    for i = 1:N
        for j = 1:model.nq
            # hands
            x_l[idx.x[i][j]] = model.jnt_range[j,1]
            x_u[idx.x[i][j]] = model.jnt_range[j,2]
        end
    end
    for i = 1:(N-1)
        for j = 1:model.nu
            x_l[idx.u[i][j]] = model.actuator_ctrlrange[j,1]
            x_u[idx.u[i][j]] = model.actuator_ctrlrange[j,2]
        end
        # for j in [2,5,4]
        #     x_l[idx.u[i][j]] = 0
        #     x_u[idx.u[i][j]] = model.actuator_ctrlrange[j,2]
        # end
    end
    

    # inequality constraints
    c_l = zeros(0)
    c_u = zeros(0)
    # c_l = -0.01*ones(idx.nx+3+idx.nx*(N-1))
    # c_u = 0.01*ones(idx.nx+3+idx.nx*(N-1))
    # initial guess 
    z0 = randn(idx.nz)
    z0[idx.x[1]] .= xic
    for i = 2:N
        z0[idx.x[i]] .= xic + ((xg_physics-xic)/(N))*(i)
    end
    println(length(robohand_inequality_constraints(params,z0)))
    println(idx.nx)
    for i = 1:N-1
        for j = 1:model.nu
            z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        end
        # for j in [2,5,4]
        #     z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        # end
    end
    # diff type
    diff_type = :auto 
    # diff_type = :finite
    
    
    Z = fmincon(robohand_cost,robohand_equality_constraints,robohand_inequality_constraints,
                x_l,x_u,c_l,c_u,z0,params, diff_type;
                tol = 1e-3, c_tol = 1e-2, max_iters = 100, verbose = verbose)
    
    # pull the X and U solutions out of Z 
    X = [Z[idx.x[i]] for i = 1:N]
    U = [Z[idx.u[i]] for i = 1:(N-1)]
    return X, U, t_vec, params
end

solve_finger_moving (generic function with 1 method)

In [399]:
X, U, t_vec, params = solve_finger_moving(verbose=true)

([[6.55602159868499e-8, 4.747392043212389e-9, 6.55602159868499e-8, 6.55602159868499e-8, 4.747392043212389e-9, 6.55602159868499e-8, 4.747392043212389e-9, 6.55602159868499e-8, 6.55602159868499e-8, 4.747392043212389e-9  …  -0.16775113564068206, -0.03039697527948987, -0.8912216587082428, -0.004234131564461016, -4.2233680988673, 0.45938440643413353, 0.34952271077786434, 0.17690537583294683, 0.045559012633187956, 0.010943869295629263], [3.6949852164453123e-8, -1.1102696104330678e-9, 3.694985216445313e-8, 3.694985216445313e-8, -1.1102696104330678e-9, 3.694985216445313e-8, -1.1102696104330678e-9, 3.694985216445313e-8, 3.694985216445313e-8, -1.1102696104330678e-9  …  -0.28154710332910954, -0.05235899848240928, -0.8110357109204999, 0.01977546798011555, -3.337206498625019, -0.6216328547242577, 0.2631988823317182, 4.657386136132878, 1.4451625028982087, 1.5288603641229404], [2.6433554413786583e-8, -1.3413472723083575e-9, 2.6433554413786583e-8, 2.6433554413786583e-8, -1.3413472723083575e-9, 2.643355

In [400]:
X

11-element Vector{Vector{Float64}}:
 [6.55602159868499e-8, 4.747392043212389e-9, 6.55602159868499e-8, 6.55602159868499e-8, 4.747392043212389e-9, 6.55602159868499e-8, 4.747392043212389e-9, 6.55602159868499e-8, 6.55602159868499e-8, 4.747392043212389e-9  …  -0.16775113564068206, -0.03039697527948987, -0.8912216587082428, -0.004234131564461016, -4.2233680988673, 0.45938440643413353, 0.34952271077786434, 0.17690537583294683, 0.045559012633187956, 0.010943869295629263]
 [3.6949852164453123e-8, -1.1102696104330678e-9, 3.694985216445313e-8, 3.694985216445313e-8, -1.1102696104330678e-9, 3.694985216445313e-8, -1.1102696104330678e-9, 3.694985216445313e-8, 3.694985216445313e-8, -1.1102696104330678e-9  …  -0.28154710332910954, -0.05235899848240928, -0.8110357109204999, 0.01977546798011555, -3.337206498625019, -0.6216328547242577, 0.2631988823317182, 4.657386136132878, 1.4451625028982087, 1.5288603641229404]
 [2.6433554413786583e-8, -1.3413472723083575e-9, 2.6433554413786583e-8, 2.6433554413786583e-

In [401]:
U

10-element Vector{Vector{Float64}}:
 [-0.16938825881845074, -0.18647242416771426, -0.00011235024463807266, 0.3643164418378407, 0.3032275676810508, -0.14958262292977345, 0.029993200522075394]
 [-0.16917667537893796, -0.15550214810465296, 0.0003052544471976928, 0.4495310154312543, 1.0128403857088124, -0.14958262292977467, 0.029993200522075394]
 [-0.169618359471138, -0.3380931313053855, -0.00041525366169325306, 0.4888151838248504, 1.2536446683690612, -0.14958262292977303, 0.029993200522075394]
 [-0.16892670590649309, 0.3896586195522965, 0.0004226996933436513, 0.3993408752124926, 0.47867893775758713, -0.14958262292977445, 0.029993200522075394]
 [-0.16986289032498875, -0.2668215235500853, -0.000343845628401506, 0.2973302531683051, 0.6918172984670423, -0.14958262292977328, 0.029993200522075394]
 [-0.16873607321694617, -0.07957300121092824, 0.00022007950443492979, 0.6902875562785222, 1.4273200322814699, -0.1495826229297747, 0.029993200522075394]
 [-0.16994202611129713, 0.05307897281746685, -9

In [402]:
# instantiate model and data
model = load_model("piano_with_one_shadow_hand_small/scene.xml")
data = init_data(model)

# reset the model and data j in [2,5,4]
        #     z0[idx.u[i][j]] = rand(Uniform(model.actuator_ctrlrange[j,1],model.actuator_ctrlrange[j,2]))
        # end
tf = 0.1
t_vec = 0:dt:tf
N = length(t_vec)
states = zeros(nx, N)
ctrl_states = zeros(model.nu, N)
reset!(model, data)
step!(model, data)
states[:,1] = get_physics_state(model, data)

for t in 1:N-1
    data.ctrl .= U[t][:]
    step!(model, data)
    states[:,t+1] = get_physics_state(model, data)
end
println(data.geom_xpos[finger_geom_indices[1],:])
states3 = zeros(nx, N)
for t in 1:N
    states3[:,t] = X[t][:]
end

reset!(model, data)
nx = model.nq + model.nv + model.na # State vector dimension

states2 = zeros(nx, N)
states2[:,1] = get_physics_state(model, data)
U2 = zeros(model.nu)
U2[4] = 0.664
U2[5] = 1.1
U2[2] = 0.21
for t in 1:N-1
    data.ctrl .= U2
    step!(model, data)
    states2[:,t+1] = get_physics_state(model, data)
end
println(data.geom_xpos[finger_geom_indices[1],:])

In [403]:
init_visualiser()
visualise!(model, data, trajectories = [states])


2886r 6.0283676e+00 4.07e+00 2.86e+04   0.4 2.43e-01    -  1.00e+00 1.00e+00h  1
2887r 6.0572511e+00 2.85e+00 4.15e+04   0.4 4.97e-01    -  1.00e+00 1.00e+00h  1
2888r 6.0628306e+00 2.72e+00 1.74e+05   0.4 1.85e-01    -  8.86e-01 6.13e-02h  4
2889r 6.0703610e+00 2.57e+00 2.80e+05   0.4 2.05e-01    -  1.00e+00 5.62e-02h  5
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2890r 6.1306098e+00 1.53e+00 5.22e+05   0.4 1.86e-01    -  9.86e-01 5.00e-01h  2
2891r 6.1415871e+00 1.37e+00 2.03e+04   0.4 1.53e-01    -  1.00e+00 9.88e-02h  4
